## TMA4215 - Exercise 2 ## 
*Group: Hanna Heshmati Rød, Karine Austbø Grande and Thea Boge*

In [7]:
import numpy as np
import scipy.linalg
import time

#### Problem 1
#### a)
Show that: $cond_{||.||}   (\alpha A) = cond_{||.||}$ for $\alpha \in \mathbb{R} \setminus \{0\}$

$ cond_{||.||}(\alpha A) =||\alpha A||~||(\alpha A)^{-1}|| = ||\alpha A||~||(\alpha)^{-1} (A)^{-1} ||= |\alpha| ||A|| |\alpha ^{-1}| ||A^{-1}|| = ||A||~||A^{-1}|| = cond_{||.||}(A)$

#### b)
Show that: $cond_{||.||} (AB) \leq cond_{||.||}(A) \cdot cond_{||.||}(B)$ for an invertible $B \in \mathbb{R}^{dxd}$

$cond_{||.||}(AB) = ||AB||~||(AB)^{-1}|| = ||AB||~||A^{-1}B^{-1}|| \leq ||A||~||A^{-1}||~||B||~||B^{-1}|| = cond_{||.||}(A) \cdot cond_{||.||}(B) $

The inequality is from Theorem 1.3.

#### c)
Show that: $cond_{||.||}(A) \geq \rho(A) \rho(A^{-1}) $

From Theorem 1.4 we have that $\rho (A) \leq ||A|| ~~ \forall A \in \mathbb{C}^{2x2} $, where $||.||$ is a consistent matrix norm.

We then have: 


$cond_{||.||} = ||A||~||A^{-1}|| \geq \rho(A) \rho(A^{-1})$

#### d)
Show that: $cond_{||.||_{2}} (Q) = 1$  for an orthogonal matrix Q.

For an orthogonal matrix we have $Q^{-1} = Q^{T}$

$cond_{||.||_{2}} (Q) = ||Q||_{2} ||Q^{-1}||_{2} = ||Q||_{2} ||Q^{T}||_{2} = \sqrt{\rho(Q^{T}Q)} \cdot \sqrt{\rho(QQ^{T})} = \sqrt{\rho(I)} \cdot \sqrt{\rho(I)} = 1 \cdot 1 = 1$

#### Problem 2

#### a)

In [ ]:
def mylu(A):
    n = A.shape[0]
    P = np.arange(n)

    LU = A.copy()
    
    for k in range(n):
        col = np.abs(LU[P[k:], k]) #extracting column k from the diagonal and down
        P_l = np.argmax(col) + k #index of the maximum value of the column

        if P_l != k:
            P[k], P[P_l] = P[P_l], P[k] #swapping P_k by P_l

        #Perform elimination
        for i in range(k+1, n):
            multiplier = LU[P[i],k] / LU[P[k],k] #find multipliers

            for j in range(k+1, n):
                LU[P[i],j] -= multiplier * LU[P[k],j]

    return LU, P

#### b)

In [ ]:
def mylu_test(A):

    P, LU = mylu(A)

    P_matrix = np.eye(A.shape[0])[P]

    L = np.eye(A.shape[0])
    U = np.zeros_like(A)

    for i in range(A.shape[0]):
        for j in range(A.shape[0]):
            if i > j:
                L[i, j] = LU[P[i], j]
            else:
                U[i, j] = LU[P[i], j]

    # Do the factorization using SciPy
    P_scipy, L_scipy, U_scipy = scipy.linalg.LU(A)

    print("Our L:\n", L)
    print("SciPy L:\n", L_scipy)

    print("Our U:\n", U)
    print("SciPy U:\n", U_scipy)

    print("Our permutation P:\n", P_matrix)
    print("SciPy permutation P:\n", P_scipy)
    
    # Check if the decomposed matrices match
    is_close = np.allclose(L @ U, P_matrix @ A) and np.allclose(L_scipy @ U_scipy, P_scipy @ A)
    print("Are the decompositions the same? ", is_close)

In [ ]:
A = np.array([[2,5,8,7], [5,2,2,8], [7,5,6,6], [5,4,4,8]])

print(mylu_test(A))

#### c)

#### Problem 3
#### a)
Under we will define a function that takes a lower triangular matrix $A$ and some vector $b$ to solve $Ax=b$.

In [2]:
def forward_sub(A, b):

    n = A.shape[0]
    x = np.zeros_like(b)

    for i in range(n):
        x[i] = (b[i] - np.dot(A[i, :i], x[:i])) / A[i, i]
    
    return x

#### b)
Now we will define a function that takes an upper triangular matrix $A$ and some vector $b$ to solve $Ax=b$.

In [3]:
def backward_sub(A, b):

    n = A.shape[0]
    x = np.zeros_like(b)

    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    
    return x

#### c)
Now we combine the last two parts with Problem 2 by inplementing a function for solving. Meaning it computes the LU decomposition of $A$ and then computes $Ax=b$.

In [5]:
def my_solve(A,b):

    LU, P = mylu(A) # LU factorization
    L = np.tril(LU,-1) + np.eye(A.shape[0])
    U = np.triu(LU)

    b_p = P[b]
    y = forward_sub(A,b_p)
    x = backward_sub(A,y)

    return x

#### d)

In [ ]:
n = 100
m = 200

# generating a regular square matrix A
L = np.tril(np.random.rand(n,n),-1) + np.eye(n)
U = np.triu(np.random.rand(n,n))

A = L @ U

# generating m right hand sides
b_k = np.random.rand(n,m)

# 1) calling m times my_solve(A,b) once for each b_k
start = time.time()
for i in range(m):
    my_solve(A,B[:,i])
end = time.time()-start

#### Problem 4
Compute the Cholesky decomposition of the followin symmetric positive define matrix

$A = \left[ \begin{matrix} 3 & 9 & 6 
\\ 9 & 29 & 16 
\\ 6 & 16 & 17 \end{matrix} \right]$



From Theorem 3.6: Let $A \in \mathbb{R}^{nxn}$ be a symmetric and positive definite matrix. Then there exists a unique upper triangulare matrix H with positive diagonal entries such that

 $A = H H^{T}$ 

 The entries $h_{ij}$ of $H^{T}$ can be computed as follows:

 $h_{11} = \sqrt{a_{11}}$ and, for $ i = 2, ..., n$

 $h_{ij} = (a_{ij} - \sum_{k=1}^{j-1} h_{ik} h_{jk})/h_{jj}$, $j = 1, ..., i-1$

 $h_{ii} =(a_{ii} - \sum_{k=1}^{i-1} h_{ik}^{2})^{1/2}$


 We can then calculate the following coefficients for them matrices $H$ and $H^{T}$:

$h_{11} = \sqrt{3}$

$h_{21} = \dfrac{a_{21}}{h_{11}} = \dfrac{9}{\sqrt{3}} = 3 \sqrt{3}$

$h_{22} = \sqrt{a_{22} - h_{21}^{2}} = \sqrt{29-27} = \sqrt{2}$

$h_{31} = \dfrac{a_{31}}{h_{11}} = \dfrac{6}{\sqrt{3}} = 2 \sqrt{3}$

$h_{32} = \dfrac{a_{32} - (h_{31} h_{21})} {h_{22}} = \dfrac{16 - (2 {\sqrt{3} 3 \sqrt{3}})}{\sqrt{2}} = -\sqrt{2}$

$h_{33} = \sqrt{a_{33} - (h_{31}^{2} + h_{32}^{2})} = \sqrt{17 - ((2\sqrt{3})^{2} + (-\sqrt{2})^{2})} = \sqrt{3}$

We get the matrices:

$H^{T} = \left[ \begin{matrix} 
\sqrt{3} & 3 \sqrt{3} & 2 \sqrt{3} 
\\ 0 & \sqrt{2} & -\sqrt{2}
\\ 0 & 0 & \sqrt{3} \end{matrix} \right]$

$H = \left[ \begin{matrix} 
\sqrt{3} & 0  & 0
\\ 3 \sqrt{3} & \sqrt{2} & 0
\\ 2 \sqrt{3} & -\sqrt{2} & \sqrt{3} \end{matrix} \right]$